In [1]:
from crewai import Agent
from textwrap import dedent
from langchain.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI

## Cheatsheet on how to start defining the agents


In [2]:
'''
Creating agents cheatsheet:
- Think like a boss, work backwards from the goal and think about what employees you want to hire
- Define the captain of the crew who orient the other agents towards the goal
- Define which experts the captain needs to communicate with and delegate tasks to.
- Build a top down approach

Goal:
- Create a 7 day itinerary with detailed per day plans., including budget packing suggestions and safety tips.

Captain:
- Expert travel agent

Employees:
- City selection expert
- Local tour guide expert

Notes:
- IMPORTANT: Agent should be *results* driven and have a clear goal in mind
- Role is their job title
- Goals should be actionable
- Backstory should be their resume
'''

'\nCreating agents cheatsheet:\n- Think like a boss, work backwards from the goal and think about what employees you want to hire\n- Define the captain of the crew who orient the other agents towards the goal\n- Define which experts the captain needs to communicate with and delegate tasks to.\n- Build a top down approach\n\nGoal:\n- Create a 7 day itinerary with detailed per day plans., including budget packing suggestions and safety tips.\n\nCaptain:\n- Expert travel agent\n\nEmployees:\n- City selection expert\n- Local tour guide expert\n\nNotes:\n- IMPORTANT: Agent should be *results* driven and have a clear goal in mind\n- Role is their job title\n- Goals should be actionable\n- Backstory should be their resume\n'

## Generating the custom agents that we need to perform our task

In [3]:
class CustomAgents:
    def __init__(self):
        self.OpenAIGPT35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
        self.OpenAIGPT4 = ChatOpenAI(model_name="gpt-4", temperature=0.7)
        self.Ollama = Ollama(model="openhermes")

    def expert_travel_agent(self):
        return Agent(
            role="Expert Travel Agent",
            # Do the back story last, because the backstory should support the goal
            backstory=dedent(f"""Expert in travel planning and logistics. I have decades of experience making travel iteneraries. """),
            goal=dedent(f"""Create a 7 day travel itinerary with detailed per-day plans, 
                        include budget packing suggestions and safety tips.
                        """),
            # tools=[tool_1, tool_2],
            allow_delegation=False,
            verbose=True,
            #CUSTOMISE THIS
            llm=self.OpenAIGPT35,
        )

    def city_selection_expert(self):
        return Agent(
            role="City Selection Expert",
            backstory=dedent(f"""Expert at analysing travel data to pick ideal destinations"""),
            goal=dedent(f"""Select the best cities based on weather, season, prices and traveler interests.
                        """),
            # tools=[tool_1, tool_2],
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT35,
        )
    
    def local_tour_guide(self):
        return Agent(
            role="Local Tour Guide",
            backstory=dedent(f"""Knowledgeable local guide with extensive information."""),
            goal=dedent(f"""Provide the BEST insights about the selected city"""),
            # tools=[tool_1, tool_2],
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT35,
        )


## Creating the tools we need to help our agents do the job well
- These should be under a tools file

In [6]:
from langchain.tools import tool
#calculator tools
class CalculatorTools():
    @tool("Make a calculation")
    def calculate(operation):
        '''
        Useful for performing any mathematical calculations, like sum minus, multiplication,
        The input to this tool should be a mathematical expression, a couple of examples are '200x7' or '5000/2*10'
        '''
        try:
            return eval(operation)
        except SyntaxError:
            return "ERROR: Invalid syntax in mathematical expression"        

In [ ]:
# Search tools
'''
Refer to https://serper.dev/playground to get source code for the API calling or to get your key
'''

import json
import os
import requests
from dotenv import load_dotenv
from langchain.tools import tool

load_dotenv()
SERPER_KEY = os.environ.get('SERPER_API_KEY')


class SearchTools():

  @tool("Search the internet")
  def search_internet(query, num_results):
    """Useful to search the internet
    about a a given topic and return relevant results"""
    
    # Set API endpoint
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': SERPER_KEY,
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    # check if there is an organic key
    if 'organic' not in response.json():
      return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
    else:
      results = response.json()['organic']
      string = []
      for result in results[:num_results]:
        try:
          string.append('\n'.join([
              f"Title: {result['title']}", f"Link: {result['link']}",
              f"Snippet: {result['snippet']}", "\n-----------------"
          ]))
        except KeyError:
          next

      return '\n'.join(string)
  
